# Imputing Missing Values - LDS Model
* Perform Kalman filter to impute missing data in lab test features
* Uniformly select 200 (1% of training set) KalmanFilter models to impute test set
* Impute vital signal data using nearest values and zeros; drop feature `EtCO2` because of all missing
* Drop features `Unit1` and `Unit2` in demographics data; impute closest values or zeros for missing values

## Assistant Functions and KalmanFilter Subclass

In [1]:
# function that prints model parameters
def print_parameters(kf_model, need_params=None, evals=False):
    """
    Function that prints out the parameters for a Kalman Filter
    @param - kf_model : the model object
    @param - need_params : a list of string
    """
    if evals:
        if need_params is None:
            need_params1 = ['transition_matrices', 'transition_covariance',
                            'observation_covariance', 'initial_state_covariance']
            need_params2 = ['observation_matrices', 'initial_state_mean']
        for param in need_params1:
            tmp = np.linalg.eig(getattr(kf_model, param))[0]
            print("{0} = {1}, shape = {2}\n".format(param, tmp, tmp.shape))
        for param in need_params2: 
            print("{0} = {1}, shape = {2}\n".format(param, getattr(kf_model, param),getattr(kf_model, param).shape))
    else:
        if need_params is None:
            need_params = ['transition_matrices', 'observation_matrices',
                           'transition_covariance', 'observation_covariance', 
                            'initial_state_mean', 'initial_state_covariance']
        for param in need_params: 
            print("{0} = {1}, shape = {2}\n".format(param, getattr(kf_model, param), getattr(kf_model, param).shape))

In [2]:
# function to impute missing values
def impute_missing(C_matrix, state_mean, X, lower, upper):
    
    """
    Impute missing values in lab data. If imputed values are
    out of the range of (lower, upper), impute zero instead.
    
    @params: C_matrix: Numpy Array, observation matrix.
             state_mean: Numpy Array, prediected state mean;
                         obtained by transition matrix and state
                         mean of previous time step.
             X: Numpy Array, observation data.
             lower: Float, lower bound of a feature.
             upper: Float, upper bound of a feature.
    
    @return: Numpy Array, clean observation data.
    """
    
    # get the indices of missing values
    nan_indices = np.argwhere(np.isnan(X))
    nan_indices = sum(nan_indices.tolist(), [])
    
    # predict obs from state mean
    x_pred = np.dot(C_matrix, state_mean)
    
    for i in range(len(x_pred)):
        if x_pred[i] < lower[i] or x_pred[i] > upper[i]:
            x_pred[i] = 0
    
    current_observed_data = X       
    
    if len(nan_indices) != 0:
        current_observed_data[nan_indices] = x_pred[nan_indices]    # impute missing values
        X[nan_indices] = x_pred[nan_indices]    # impute original data
    return current_observed_data

In [3]:
def concat_patients(train_dir, patient_list):
    
    """
    concatenate individual patient dataframe for
    the training, valid, or other dataframes to define
    lower and upper bound of each feature.
    
    @params: train_dir: String, data folder
             patient_list: list of Strings, individual patient files
    
    @return: dataframe, concatenated training or valid data
    """
    
    # read in first patient in patient list
    p = pd.read_csv(train_dir + '/' + patient_list[0], sep = "|")
    p['patient_id'] = patient_list[0][1:7]

    for i in range(1, len(patient_list)):
        p_n = pd.read_csv(train_dir + '/' + patient_list[i], sep = "|")
        p_n['patient_id'] = patient_list[i][1:7]
        p = p.append(p_n)
    return p

In [4]:
def impute_dataset(load_dir, patients, impute_models, save_dir, attributes, labels):
    
    """
    function that imputes missing values for valid and test datasets.
    
    @params load_dir: String, the directory of raw data
            patients: List of String, patient filenames
            impute_models: KalmanFilter Object, used to impute missing
                           values in raw data
            save_dir: String, the directory to save clean data
            attributes: List of String, predictors
    @return: None
    """
    
    # impute patient by patient
    for p in patients:
        df = pd.read_csv(load_dir + '/' + p, sep = "|")[attributes]
        labels_df = pd.read_csv(load_dir + '/' + p, sep = "|")[labels]

        # LDS impute missing values
        results = []    # save clean dataframes from all models
        for model in impute_models:
            df_copy = df.copy()
            col_names = df.columns[:]
            data_np = df_copy.to_numpy()
            ts_imputed_mu, ts_imputed_cov = model.filter_missing(data_np)
            results.append(pd.DataFrame(data_np, columns=col_names))
        df_clean = pd.concat(results).groupby(level=0).mean()
        df_clean[labels] = labels_df
        df_clean['patient_id'] = p[1:7]
        df_clean = df_clean.drop(['Unit1', 'Unit2', 'EtCO2'], axis=1)

        # save clean patient test data
        df_clean.to_csv(save_dir + p, sep='|')

        print(p)

In [5]:
# create a subclass of KalmanFilter
# define a new filter method to impute missing vals
from pykalman import KalmanFilter
class MyKalmanFilter(KalmanFilter):
    
    """
    Subclass of KalmanFilter.
    Rewrite filter method to filter missing values in data.
    """
    
    def __init__(self, transition_matrices=None, observation_matrices=None,
                 transition_covariance=None, observation_covariance=None,
                 transition_offsets=None, observation_offsets=None,
                 initial_state_mean=None, initial_state_covariance=None,
                 random_state=None, em_vars=['transition_covariance', 'observation_covariance',
                                             'initial_state_mean', 'initial_state_covariance'],
                 n_dim_state=None, n_dim_obs=None):
        super().__init__(transition_matrices=None, observation_matrices=None,
                         transition_covariance=None, observation_covariance=None,
                         transition_offsets=None, observation_offsets=None,
                         initial_state_mean=None, initial_state_covariance=None,
                         random_state=None, em_vars=['transition_covariance', 'observation_covariance',
                                             'initial_state_mean', 'initial_state_covariance'],
                         n_dim_state=None, n_dim_obs=None)
        
        self.n_dim_state = n_dim_state
        self.n_dim_obs = n_dim_obs
    
    def filter_missing(self, X):
        
        n_example, observed_dim = X.shape
        
        # create holders for outputs
        filtered_state_means = np.zeros( (n_example, self.n_dim_state) )
        filtered_state_covariances = np.zeros( (n_example, self.n_dim_state, self.n_dim_state) )
        
        # the first state mean and state covar is the initial epectation
        filtered_state_means[0] = self.initial_state_mean
        filtered_state_covariances[0] = self.initial_state_covariance
        
        # initialize internal variables
        current_state_mean = self.initial_state_mean.copy()
        current_state_covar = self.initial_state_covariance.copy()
        current_observed_data = impute_missing(self.observation_matrices, current_state_mean, X[0,:], lower, upper)
   

        for i in range(1, n_example): 
            
            current_observed_data = X[i,:]
            
            # run a single step forward filter
            # prediction step
            predicted_state_mean = np.dot(self.transition_matrices, current_state_mean)
            predicted_state_cov = np.matmul(np.matmul(self.transition_matrices, current_state_covar),
                                            np.transpose(self.transition_matrices)) + self.transition_covariance
            
            # predict observation
            current_observed_data = impute_missing(self.observation_matrices,
                                                   predicted_state_mean, current_observed_data, lower, upper)
            
            # observation step
            innovation = current_observed_data - np.dot(self.observation_matrices, predicted_state_mean)
            innovation_covariance = np.matmul(np.matmul(self.observation_matrices, predicted_state_cov),
                                              np.transpose(self.observation_matrices)) + self.observation_covariance
            
            # update step
            kalman_gain = np.matmul(np.matmul(predicted_state_cov, np.transpose(self.observation_matrices)),
                                    np.linalg.inv(innovation_covariance))
            current_state_mean = predicted_state_mean + np.dot(kalman_gain, innovation)
            current_state_covar = np.matmul( (np.eye(current_state_covar.shape[0])- 
                                              np.matmul(kalman_gain,self.observation_matrices)), predicted_state_cov)
            # populate holders
            filtered_state_means[i, :] = current_state_mean
            filtered_state_covariances[i, :, :] = current_state_covar
        
        return filtered_state_means, filtered_state_covariances
    
    def import_param(self, kf_model):
        """
        Method that copies parameters from a trained Kalman Model
        @param kf_model: a Pykalman object
        """
        need_params = ['transition_matrices', 'observation_matrices', 'transition_covariance', 
                  'observation_covariance', 'initial_state_mean', 'initial_state_covariance']
        for param in need_params:
            setattr(self, param, getattr(kf_model, param))

## Load the data
We begin by loading the standard modules.

In [6]:
# import packages
import pandas as pd
import os
import numpy as np

In [7]:
# list out lab test features for imputation
labs=['BaseExcess','HCO3','FiO2','pH','PaCO2','SaO2','AST','BUN','Alkalinephos','Calcium','Chloride','Creatinine','Bilirubin_direct','Glucose','Lactate',
      'Magnesium','Phosphate','Potassium','Bilirubin_total','TroponinI','Hct','Hgb','PTT','WBC','Fibrinogen','Platelets']

# list out vital signal features for imputation
vitals = ['HR','O2Sat','Temp','SBP','MAP','DBP','Resp','EtCO2']

# list out demographic features for imputation
demogs = ['Age','Gender','Unit1','Unit2','HospAdmTime','ICULOS']

# labels
labels = ['SepsisLabel']

# attributes
attributes = [vitals, labs, demogs]

In [8]:
# create a list of patient file names
train_dir = './data/raw_data/training/'
tr_patients = [p for p in sorted(os.listdir(train_dir))]
len(tr_patients)

14236

In [9]:
# concat all patients to get lower and upper bound for each lab feature
# import pickle
# with open('../raw_data.pickle', 'rb') as f:
#     data = pickle.load(f)

tr_concat_raw = pd.read_csv('./data/raw_data/train_concate.csv', sep = '|')

des = tr_concat_raw[sum(attributes,[])].describe()
IQR = 1.5 * (des.loc['75%'] - des.loc['25%'])    # interquartile range
upper = des.loc['75%'] + IQR
lower = des.loc['25%'] - IQR

## Impute missing values

### Training data

Uniformly save 200 `KalmanFilters`. Use their parameters updated by `EM` to impute each test patient missing values. Average the 200 Dataframes of each test patient and export the only one averaged Dataframe for each patient.

In [ ]:
# uniformly select 200 training patients
# save the kf models they use to impute missing values
np.random.seed(0)
indices_select = np.sort(np.random.uniform(0, len(tr_patients), 200)).astype(int)
patient_select = np.array(tr_patients)[indices_select]

Impute missing values using observation prediction in `LDS` `filter` function. Save the `KalmanFilter` models used to impute the selected 200 patients.

In [ ]:
# define kalman filter model
attributes_list = sum(attributes,[])    #attributes of dataframe
n_dim_state = len(attributes_list)
n_dim_obs = len(attributes_list)
kf = MyKalmanFilter(n_dim_state=n_dim_state, n_dim_obs=n_dim_obs)
kfs = []

for p in tr_patients[:2]:
    df = pd.read_csv(train_dir + '/' + p, sep = "|")[attributes_list]
    labels_df = pd.read_csv(train_dir + '/' + p, sep = "|")[labels]
    
    # LDS impute data
    df_copy = df.copy()
    col_names = df.columns[:]
    data_np = df_copy.to_numpy()

    # set initial paramters for the patient
    kf.transition_matrices = np.diag(np.random.random(size = n_dim_state)*0.5)
    kf.transition_covariance = np.diag(np.random.random(size = n_dim_obs))
    kf.observation_matrices = np.diag(np.random.random(size = n_dim_state)*0.1) 
    kf.observation_covariance = np.diag(np.random.random(size = n_dim_obs)) 
    kf.initial_state_mean = np.zeros(kf.n_dim_state)
    kf.initial_state_covariance = np.diag(np.random.random(size = n_dim_state)*0.1)

    filtered_imputed_mu, filtered_imputed_cov = kf.filter_missing(data_np)
    smoothed_state_mean, smoothed_state_cov = kf.smooth(data_np)
    kf.em(data_np, n_iter=5)    
    filtered_imputed_mu, filtered_imputed_cov = kf.filter_missing(data_np)
    
    # create copy of current kf model and save to impute test data
    select_kf = MyKalmanFilter(n_dim_state=n_dim_state, n_dim_obs=n_dim_obs)
    select_kf.import_param(kf)
    if p in patient_select:
        kfs.append(select_kf)

    df_clean = pd.DataFrame(data_np, columns=col_names)
    df_clean[labels] = labels_df
    df_clean['patient_id'] = p[1:7]
    df_clean = df_clean.drop(['Unit1', 'Unit2', 'EtCO2'], axis=1)
    
    # save new patient data
    save_path = './data/LDS2/train/'
    df_clean.to_csv(save_path + p, sep='|')               
    
    print(p)

### Valid dataset
Impute missing values in the test data using the 200 saved model above. Each model will generate a new dataframe for each patient. So, we save the average values across 200 dataframes for each patient as his/her clean data.

In [29]:
# create a patient list for valid data
valid_dir = './data/raw_data/validation/'
valid_save_dir = './data/LDS/valid/'
vld_patients = sorted(os.listdir(valid_dir))

# impute valid dataset
impute_dataset(valid_dir, vld_patients, kfs, valid_save_dir, sum(attributes,[]), labels)

p000002.psv
p000004.psv
p000005.psv
p000007.psv
p000014.psv
p000015.psv
p000023.psv
p000026.psv
p000031.psv
p000035.psv
p000053.psv
p000054.psv
p000068.psv
p000073.psv
p000083.psv
p000092.psv
p000098.psv
p000100.psv
p000108.psv
p000109.psv
p000115.psv
p000118.psv
p000119.psv
p000121.psv
p000139.psv
p000144.psv
p000151.psv
p000164.psv
p000169.psv
p000171.psv
p000176.psv
p000193.psv
p000196.psv
p000199.psv
p000201.psv
p000202.psv
p000204.psv
p000209.psv
p000217.psv
p000218.psv
p000219.psv
p000221.psv
p000222.psv
p000249.psv
p000256.psv
p000258.psv
p000260.psv
p000273.psv
p000274.psv
p000277.psv
p000278.psv
p000280.psv
p000283.psv
p000286.psv
p000287.psv
p000291.psv
p000294.psv
p000304.psv
p000311.psv
p000317.psv
p000329.psv
p000346.psv
p000347.psv
p000360.psv
p000371.psv
p000376.psv
p000380.psv
p000384.psv
p000388.psv
p000391.psv
p000394.psv
p000404.psv
p000408.psv
p000416.psv
p000417.psv
p000418.psv
p000424.psv
p000438.psv
p000439.psv
p000446.psv
p000454.psv
p000456.psv
p000474.psv
p000

p004395.psv
p004403.psv
p004407.psv
p004411.psv
p004427.psv
p004428.psv
p004433.psv
p004437.psv
p004447.psv
p004455.psv
p004470.psv
p004471.psv
p004474.psv
p004480.psv
p004483.psv
p004491.psv
p004526.psv
p004533.psv
p004558.psv
p004568.psv
p004569.psv
p004577.psv
p004585.psv
p004587.psv
p004610.psv
p004611.psv
p004617.psv
p004618.psv
p004620.psv
p004622.psv
p004633.psv
p004638.psv
p004645.psv
p004649.psv
p004659.psv
p004665.psv
p004669.psv
p004670.psv
p004673.psv
p004677.psv
p004679.psv
p004686.psv
p004687.psv
p004710.psv
p004711.psv
p004717.psv
p004719.psv
p004721.psv
p004732.psv
p004739.psv
p004740.psv
p004749.psv
p004750.psv
p004753.psv
p004758.psv
p004765.psv
p004779.psv
p004781.psv
p004821.psv
p004823.psv
p004825.psv
p004827.psv
p004828.psv
p004830.psv
p004839.psv
p004843.psv
p004845.psv
p004849.psv
p004851.psv
p004852.psv
p004857.psv
p004869.psv
p004875.psv
p004885.psv
p004886.psv
p004888.psv
p004889.psv
p004896.psv
p004900.psv
p004904.psv
p004908.psv
p004909.psv
p004928.psv
p004

p008948.psv
p008954.psv
p008963.psv
p008973.psv
p008981.psv
p008984.psv
p008988.psv
p008989.psv
p008992.psv
p008994.psv
p009010.psv
p009014.psv
p009023.psv
p009045.psv
p009049.psv
p009057.psv
p009060.psv
p009062.psv
p009063.psv
p009068.psv
p009078.psv
p009091.psv
p009105.psv
p009107.psv
p009109.psv
p009113.psv
p009116.psv
p009119.psv
p009126.psv
p009129.psv
p009131.psv
p009137.psv
p009147.psv
p009161.psv
p009168.psv
p009199.psv
p009202.psv
p009205.psv
p009215.psv
p009221.psv
p009238.psv
p009243.psv
p009245.psv
p009248.psv
p009249.psv
p009254.psv
p009260.psv
p009262.psv
p009263.psv
p009268.psv
p009270.psv
p009290.psv
p009292.psv
p009301.psv
p009313.psv
p009316.psv
p009319.psv
p009322.psv
p009329.psv
p009332.psv
p009343.psv
p009345.psv
p009354.psv
p009357.psv
p009367.psv
p009384.psv
p009393.psv
p009405.psv
p009408.psv
p009422.psv
p009425.psv
p009437.psv
p009446.psv
p009453.psv
p009458.psv
p009463.psv
p009474.psv
p009481.psv
p009494.psv
p009496.psv
p009502.psv
p009508.psv
p009510.psv
p009

p013867.psv
p013871.psv
p013873.psv
p013879.psv
p013882.psv
p013887.psv
p013896.psv
p013900.psv
p013901.psv
p013903.psv
p013911.psv
p013917.psv
p013923.psv
p013930.psv
p013946.psv
p013947.psv
p013957.psv
p013960.psv
p013963.psv
p013964.psv
p013978.psv
p013991.psv
p013995.psv
p013998.psv
p014000.psv
p014007.psv
p014016.psv
p014020.psv
p014028.psv
p014039.psv
p014047.psv
p014049.psv
p014050.psv
p014052.psv
p014058.psv
p014069.psv
p014070.psv
p014088.psv
p014090.psv
p014092.psv
p014098.psv
p014130.psv
p014132.psv
p014136.psv
p014145.psv
p014152.psv
p014156.psv
p014166.psv
p014174.psv
p014175.psv
p014179.psv
p014182.psv
p014186.psv
p014189.psv
p014192.psv
p014196.psv
p014205.psv
p014207.psv
p014210.psv
p014228.psv
p014237.psv
p014241.psv
p014243.psv
p014247.psv
p014249.psv
p014283.psv
p014287.psv
p014289.psv
p014304.psv
p014305.psv
p014306.psv
p014324.psv
p014333.psv
p014336.psv
p014345.psv
p014354.psv
p014367.psv
p014373.psv
p014374.psv
p014376.psv
p014379.psv
p014386.psv
p014392.psv
p014

p018282.psv
p018293.psv
p018294.psv
p018301.psv
p018303.psv
p018310.psv
p018323.psv
p018324.psv
p018358.psv
p018360.psv
p018365.psv
p018368.psv
p018370.psv
p018371.psv
p018372.psv
p018373.psv
p018381.psv
p018392.psv
p018394.psv
p018406.psv
p018420.psv
p018423.psv
p018428.psv
p018430.psv
p018443.psv
p018455.psv
p018469.psv
p018470.psv
p018475.psv
p018476.psv
p018480.psv
p018489.psv
p018491.psv
p018496.psv
p018525.psv
p018527.psv
p018528.psv
p018529.psv
p018535.psv
p018545.psv
p018546.psv
p018548.psv
p018550.psv
p018555.psv
p018562.psv
p018572.psv
p018578.psv
p018584.psv
p018588.psv
p018594.psv
p018599.psv
p018606.psv
p018622.psv
p018630.psv
p018632.psv
p018633.psv
p018635.psv
p018637.psv
p018640.psv
p018643.psv
p018644.psv
p018651.psv
p018652.psv
p018654.psv
p018657.psv
p018661.psv
p018671.psv
p018675.psv
p018678.psv
p018679.psv
p018686.psv
p018688.psv
p018708.psv
p018709.psv
p018713.psv
p018716.psv
p018722.psv
p018730.psv
p018731.psv
p018737.psv
p018743.psv
p018747.psv
p018758.psv
p018

### Test dataset
Apply the same imputation strategy on valid dataset to test dataset.

In [10]:
import pickle
with open('kfs.pickle', 'rb') as f:
    kfs = pickle.load(f)

In [15]:
# create a patient list for test data
test_dir = './data/raw_data/test/'
test_save_dir = './data/LDS/test/'
ts_patients = sorted(os.listdir(test_dir))

# impute valid dataset
impute_dataset(test_dir, ts_patients, kfs, test_save_dir, sum(attributes,[]), labels)

p000017.psv
p000019.psv
p000021.psv
p000022.psv
p000027.psv
p000028.psv
p000032.psv
p000041.psv
p000048.psv
p000049.psv
p000052.psv
p000056.psv
p000061.psv
p000067.psv
p000075.psv
p000079.psv
p000081.psv
p000089.psv
p000096.psv
p000104.psv
p000110.psv
p000117.psv
p000124.psv
p000129.psv
p000154.psv
p000158.psv
p000161.psv
p000179.psv
p000186.psv
p000192.psv
p000200.psv
p000205.psv
p000207.psv
p000210.psv
p000220.psv
p000223.psv
p000228.psv
p000231.psv
p000235.psv
p000237.psv
p000240.psv
p000243.psv
p000245.psv
p000246.psv
p000247.psv
p000255.psv
p000264.psv
p000266.psv
p000295.psv
p000296.psv
p000309.psv
p000310.psv
p000314.psv
p000328.psv
p000352.psv
p000357.psv
p000365.psv
p000378.psv
p000386.psv
p000393.psv
p000396.psv
p000398.psv
p000402.psv
p000409.psv
p000412.psv
p000422.psv
p000435.psv
p000442.psv
p000447.psv
p000455.psv
p000457.psv
p000462.psv
p000463.psv
p000466.psv
p000467.psv
p000468.psv
p000484.psv
p000485.psv
p000489.psv
p000511.psv
p000519.psv
p000529.psv
p000534.psv
p000

p004436.psv
p004446.psv
p004454.psv
p004463.psv
p004472.psv
p004486.psv
p004487.psv
p004488.psv
p004489.psv
p004496.psv
p004511.psv
p004512.psv
p004518.psv
p004520.psv
p004528.psv
p004529.psv
p004532.psv
p004534.psv
p004539.psv
p004540.psv
p004548.psv
p004552.psv
p004565.psv
p004571.psv
p004575.psv
p004580.psv
p004581.psv
p004582.psv
p004593.psv
p004601.psv
p004605.psv
p004608.psv
p004609.psv
p004612.psv
p004614.psv
p004621.psv
p004627.psv
p004628.psv
p004639.psv
p004642.psv
p004655.psv
p004663.psv
p004671.psv
p004674.psv
p004682.psv
p004683.psv
p004685.psv
p004691.psv
p004692.psv
p004693.psv
p004697.psv
p004700.psv
p004705.psv
p004730.psv
p004731.psv
p004747.psv
p004752.psv
p004760.psv
p004778.psv
p004780.psv
p004785.psv
p004787.psv
p004792.psv
p004793.psv
p004795.psv
p004801.psv
p004810.psv
p004812.psv
p004816.psv
p004832.psv
p004850.psv
p004861.psv
p004879.psv
p004884.psv
p004887.psv
p004891.psv
p004902.psv
p004922.psv
p004924.psv
p004933.psv
p004937.psv
p004941.psv
p004949.psv
p004

p009075.psv
p009077.psv
p009082.psv
p009084.psv
p009089.psv
p009094.psv
p009098.psv
p009102.psv
p009104.psv
p009114.psv
p009115.psv
p009118.psv
p009125.psv
p009134.psv
p009135.psv
p009138.psv
p009140.psv
p009149.psv
p009156.psv
p009169.psv
p009172.psv
p009175.psv
p009182.psv
p009193.psv
p009198.psv
p009203.psv
p009210.psv
p009216.psv
p009219.psv
p009224.psv
p009231.psv
p009250.psv
p009251.psv
p009253.psv
p009257.psv
p009259.psv
p009264.psv
p009277.psv
p009279.psv
p009283.psv
p009287.psv
p009293.psv
p009302.psv
p009306.psv
p009311.psv
p009326.psv
p009337.psv
p009346.psv
p009352.psv
p009362.psv
p009381.psv
p009403.psv
p009409.psv
p009410.psv
p009411.psv
p009413.psv
p009416.psv
p009433.psv
p009441.psv
p009444.psv
p009459.psv
p009475.psv
p009480.psv
p009504.psv
p009507.psv
p009509.psv
p009517.psv
p009518.psv
p009530.psv
p009545.psv
p009562.psv
p009572.psv
p009575.psv
p009578.psv
p009583.psv
p009591.psv
p009594.psv
p009607.psv
p009608.psv
p009622.psv
p009624.psv
p009626.psv
p009634.psv
p009

KeyboardInterrupt: 